In [1]:
import graspologic as gp
import matplotlib.pyplot as plt
import numpy as np
from graspologic.embed import AdjacencySpectralEmbed, OmnibusEmbed
from graspologic.plot import heatmap
from graspologic.simulations import p_from_latent, sample_edges

In [5]:
from pkg.inference.vertex_position_test import *
from pkg.inference.vertex_position_test import _bootstrap, _difference_norm, _embed

In [3]:
def compute_delta(p, eps):
    delta = ((p + eps) ** 2 - p**2) / p

    return delta


def sample_latent(n, p, eps, seed=None):
    ps = np.repeat(p, n).reshape(-1, 1)
    X = np.sqrt(ps)

    delta = compute_delta(p, eps)
    delta_vec = np.zeros((n, 1))
    delta_vec[0, 0] += delta

    Y = np.sqrt(ps + delta_vec)

    return X, Y


def sample_graphs(n, p, eps, seed=None):
    X, Y = sample_latent(n=n, p=p, eps=eps, seed=seed)

    P_X = gp.simulations.p_from_latent(X)
    P_Y = gp.simulations.p_from_latent(Y)

    np.random.seed(seed)
    A = gp.simulations.sample_edges(P_X)
    B = gp.simulations.sample_edges(P_Y)

    return A, B

In [51]:
n = 100
p = 0.25
eps = np.linspace(0, 0.75, 16)

n_reps = 500

args = [dict(n=n, p=p, eps=e) for e in eps]

In [53]:
from joblib import Parallel, delayed

In [54]:
def experiment(n, p, eps, n_reps=500, n_jobs=-1):
    def worker(seed):
        A, B = sample_graphs(n, p, eps, seed)
        _, pval, _, _ = vertex_position_test(A, B, n_components=2)  # rank 2 p matrix

        return pval

    res = Parallel(n_jobs=n_jobs)(delayed(worker)(seed) for seed in range(n_reps))

    return res